In [ ]:
import os, re, math, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# =================== CONFIG ===================
FILEPATH   = "/content/fire_archive_M6_156000.csv"   # <-- Cambia por tu Excel o CSV
TARGET_COL = None                # <-- opcional: ej. "PM2.5", "frp", "brightness"
MISSING_THRESHOLD = 0.40         # columnas con >40% NA se eliminan
RANDOM_STATE = 42
# ==============================================

# ---------- Utilidades ----------
def load_any_table(path: str) -> pd.DataFrame:
    ext = os.path.splitext(path)[1].lower()
    if ext in [".xls", ".xlsx"]:
        return pd.read_excel(path)
    # CSV: primero intento estándar, luego con ';'
    try:
        return pd.read_csv(path)
    except Exception:
        return pd.read_csv(path, sep=";")

def standardize_column_names(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (df.columns
                  .str.strip()
                  .str.replace(r"\s+", "_", regex=True)
                  .str.replace(r"[^\w]", "", regex=True)
                  .str.lower())
    return df

def parse_date_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for col in df.columns:
        if df[col].dtype == "object":
            parsed = pd.to_datetime(df[col], errors="coerce")
            if parsed.notna().mean() >= 0.5:  # si la mitad o más parecen fechas
                df[col] = parsed
    return df

def basic_cleaning(df: pd.DataFrame, missing_thresh: float = 0.4) -> pd.DataFrame:
    df = df.copy()
    # 1) quitar columnas con demasiados NA
    df = df.loc[:, df.isna().mean() <= missing_thresh]
    # 2) quitar columnas constantes
    df = df.loc[:, df.nunique(dropna=True) > 1]
    # 3) intentar convertir objetos numéricos (coma decimal)
    for c in df.columns:
        if df[c].dtype == "object":
            try:
                s = df[c].str.replace(",", ".", regex=False)
                df[c] = pd.to_numeric(s, errors="ignore")
            except Exception:
                pass
    # 4) ordenar por la primera columna datetime si existe
    date_cols = [c for c in df.columns if np.issubdtype(df[c].dtype, np.datetime64)]
    if date_cols:
        df = df.sort_values(date_cols[0]).reset_index(drop=True)
    # 5) duplicados
    return df.drop_duplicates().reset_index(drop=True)

def quick_eda(df: pd.DataFrame, target: str | None):
    print("\n=== FORMA DEL DATASET ===")
    print(df.shape)
    print("\n=== TIPOS ===")
    print(df.dtypes)
    print("\n=== NULOS (proporción) ===")
    print(df.isna().mean().sort_values(ascending=False).head(20))
    if target and target in df:
        print(f"\n=== DESCRIPTIVOS de '{target}' ===")
        print(df[target].describe())

def select_target_and_features(df: pd.DataFrame, target=None):
    numeric_cols = [c for c in df.columns if np.issubdtype(df[c].dtype, np.number)]
    if not numeric_cols:
        raise ValueError("No hay columnas numéricas para modelar.")
    if target is None:
        priority = ["frp","pm25","pm2_5","pm10","o3","no2","so2","co","brightness","value","target","y"]
        lower = {c: c.lower() for c in df.columns}
        for p in priority:
            for c in numeric_cols:
                if lower[c] == p:
                    target = c; break
            if target: break
        if target is None:
            # elegir la numérica con menos NA y mayor varianza
            candidates = []
            for c in numeric_cols:
                candidates.append((c, df[c].isna().mean(), df[c].var(skipna=True)))
            candidates = sorted(candidates, key=lambda x: (x[1], -0 if pd.isna(x[2]) else -x[2]))
            target = candidates[0][0]
    features = [c for c in numeric_cols if c != target]
    # si hay demasiadas, quedarse con las 30 más correlacionadas
    if len(features) > 30:
        corr = df[features + [target]].corr(numeric_only=True)[target].abs().sort_values(ascending=False)
        features = corr.index.tolist()[1:31]
    return target, features

def build_models(feats):
    # Preprocesamiento (escalado) solo para LR
    preproc = ColumnTransformer([("num", StandardScaler(), feats)], remainder="drop")
    lr = Pipeline([("prep", preproc), ("model", LinearRegression())])
    rf = Pipeline([("model", RandomForestRegressor(n_estimators=300, random_state=RANDOM_STATE, n_jobs=-1))])
    return {"LinearRegression": lr, "RandomForest": rf}

def evaluate(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    return {
        "MAE": float(mean_absolute_error(y_true, y_pred)),
        "MSE": float(mse),
        "RMSE": float(mse ** 0.5),
        "R2": float(r2_score(y_true, y_pred))
    }

# --------- Gráficas (matplotlib puro) ---------
def plot_hist(series: pd.Series, title: str):
    plt.figure()
    series.dropna().plot(kind="hist", bins=30)
    plt.title(title); plt.xlabel(series.name if series.name else ""); plt.ylabel("Frecuencia")
    plt.show()

def plot_timeseries(df: pd.DataFrame, target: str):
    date_cols = [c for c in df.columns if np.issubdtype(df[c].dtype, np.datetime64)]
    if not date_cols: return
    col = date_cols[0]
    plt.figure()
    plt.plot(df[col], df[target])
    plt.title(f"Serie temporal de {target}"); plt.xlabel(col); plt.ylabel(target)
    plt.show()

def plot_real_vs_pred(y_true, y_pred, title: str):
    lim_min = min(np.min(y_true), np.min(y_pred))
    lim_max = max(np.max(y_true), np.max(y_pred))
    plt.figure()
    plt.scatter(y_true, y_pred, s=10)
    plt.plot([lim_min, lim_max], [lim_min, lim_max])
    plt.title(title); plt.xlabel("Real"); plt.ylabel("Predicción")
    plt.show()

def plot_residuals(y_true, y_pred):
    resid = y_true - y_pred
    plt.figure()
    plt.hist(resid, bins=30)
    plt.title("Histograma de residuos"); plt.xlabel("Residuo"); plt.ylabel("Frecuencia")
    plt.show()

def plot_feature_importance(model, feats):
    if hasattr(model, "feature_importances_"):
        imp = model.feature_importances_
        order = np.argsort(imp)[::-1]
        plt.figure(figsize=(8, max(3, len(feats) * 0.25)))
        plt.bar(range(len(feats)), imp[order])
        plt.xticks(range(len(feats)), [feats[i] for i in order], rotation=90)
        plt.title("Importancia de variables (RandomForest)")
        plt.tight_layout()
        plt.show()

# =================== EJECUCIÓN ===================
def main():
    # 1) Cargar
    df = load_any_table(FILEPATH)
    df = standardize_column_names(df)
    df = parse_date_columns(df)
    # 2) Limpiar/filtrar/ordenar
    df = basic_cleaning(df, MISSING_THRESHOLD)
    # 3) EDA rápida
    target, features = select_target_and_features(df, TARGET_COL)
    quick_eda(df, target)

    # 4) Preparar datos (quita NA en features y target)
    model_df = df[features + [target]].dropna().copy()
    X = model_df[features]        # DataFrame (para que ColumnTransformer pueda usar nombres)
    y = model_df[target].values   # numpy

    # 5) Split
    test_size = 0.2 if len(model_df) >= 50 else (0.2 if len(model_df) >= 10 else 0.4)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
                                                        random_state=RANDOM_STATE)

    # 6) Modelar
    models = build_models(features)
    results, trained = {}, {}
    for name, mdl in models.items():
        mdl.fit(X_train, y_train)
        yhat = mdl.predict(X_test)
        results[name] = evaluate(y_test, yhat)
        trained[name] = mdl

    # 7) Elegir mejor por R2
    best_name = max(results, key=lambda k: results[k]["R2"])
    best_model = trained[best_name]

    # 8) Predicciones y guardado
    preds_all = best_model.predict(model_df[features])
    out = model_df.copy()
    out["y_pred"] = preds_all
    out["residual"] = out[target] - out["y_pred"]

    clean_path = "dataset_limpio.csv"
    preds_path = "predicciones.csv"
    df.to_csv(clean_path, index=False)
    out.to_csv(preds_path, index=False)

    print("\n=== RESUMEN ===")
    print(f"Target: {target}")
    print(f"Features ({len(features)}): {features[:10]}{'...' if len(features)>10 else ''}")
    print("Métricas:")
    for m, sc in results.items():
        print(f"  {m}: {sc}")

    print(f"\nArchivos guardados:\n - {os.path.abspath(clean_path)}\n - {os.path.abspath(preds_path)}")

    # 9) Gráficas
    plot_hist(model_df[target], f"Histograma de {target}")
    plot_timeseries(df, target)           # si hay fechas
    y_test_pred = best_model.predict(X_test)
    plot_real_vs_pred(y_test, y_test_pred, f"Real vs Predicción ({best_name})")
    plot_residuals(y_test, y_test_pred)
    if best_name == "RandomForest":
        plot_feature_importance(best_model.named_steps["model"], features)

if __name__ == "__main__":
    main()


/tmp/ipython-input-2807978596.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(df[col], errors="coerce")
/tmp/ipython-input-2807978596.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(df[col], errors="coerce")
/tmp/ipython-input-2807978596.py:62: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(s, errors="ignore")



=== FORMA DEL DATASET ===
(1182272, 13)

=== TIPOS ===
latitude             float64
longitude            float64
brightness           float64
scan                 float64
track                float64
acq_date      datetime64[ns]
acq_time               int64
satellite             object
confidence             int64
bright_t31           float64
frp                  float64
daynight              object
type                   int64
dtype: object

=== NULOS (proporción) ===
latitude      0.0
longitude     0.0
brightness    0.0
scan          0.0
track         0.0
acq_date      0.0
acq_time      0.0
satellite     0.0
confidence    0.0
bright_t31    0.0
frp           0.0
daynight      0.0
type          0.0
dtype: float64

=== DESCRIPTIVOS de 'frp' ===
count    1.182272e+06
mean     6.701266e+01
std      2.152388e+02
min     -5.920000e+01
25%      1.090000e+01
50%      2.250000e+01
75%      5.310000e+01
max      1.437650e+04
Name: frp, dtype: float64
